In [1]:
from backtesting.test import SMA

/opt/homebrew/lib/python3.11/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/opt/homebrew/lib/python3.11/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


In [2]:
import pandas as pd
from backtesting.lib import SignalStrategy, TrailingStrategy


class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

In [3]:
data = pd.read_csv("SBIN.csv", parse_dates=True, index_col="DATE")
# rename columns to first letter caps
data.columns = [col.capitalize() for col in data.columns]
# sort by date
data = data.sort_index()

data

,Series,Open,High,Low,Prev. close,Ltp,Close,Vwap,52w h,52w l,Volume,Value,No of trades,Symbol
DATE,,,,,,,,,,,,,,
2002-01-01,EQ,183.00,184.35,182.50,181.45,184.0,183.45,183.30,275.90,140.05,153925,2.821462e+07,0,SBIN
2002-01-02,EQ,184.75,189.50,184.75,183.45,189.5,188.60,187.16,275.90,140.05,213441,3.994780e+07,0,SBIN
2002-01-03,EQ,191.00,200.90,189.05,188.60,199.5,199.10,195.85,275.90,140.05,589106,1.153781e+08,0,SBIN
2002-01-04,EQ,200.55,204.35,200.00,199.10,202.0,202.70,202.57,275.90,140.05,397013,8.042326e+07,0,SBIN
2002-01-07,EQ,204.50,204.50,193.10,202.70,194.0,194.50,197.25,275.90,140.05,218259,4.305251e+07,0,SBIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-05,EQ,588.50,589.80,586.00,587.20,587.0,587.20,587.86,629.55,430.70,12920507,7.595474e+09,143514,SBIN
2023-06-06,EQ,588.00,588.90,583.00,587.20,586.0,585.35,585.23,629.55,430.70,11199940,6.554543e+09,140444,SBIN
2023-06-07,EQ,588.00,590.60,586.00,585.35,588.8,589.20,588.49,629.55,430.70,11113781,6.540372e+09,170307,SBIN


In [4]:
from backtesting import Backtest
from backtesting.test import GOOG

bt = Backtest(data, SmaCross, commission=.002)

bt.run()
# bt.plot()

Start                     2002-01-01 00:00:00
End                       2023-06-09 00:00:00
Duration                   7829 days 00:00:00
Exposure Time [%]                   32.727954
Equity Final [$]                    2632.6263
Equity Peak [$]                    19990.5881
Return [%]                         -73.673737
Buy & Hold Return [%]              215.453802
Return (Ann.) [%]                   -6.118325
Volatility (Ann.) [%]               23.111117
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -90.157435
Avg. Drawdown [%]                  -13.847712
Max. Drawdown Duration     3125 days 00:00:00
Avg. Drawdown Duration      550 days 00:00:00
# Trades                                  113
Win Rate [%]                        39.823009
Best Trade [%]                      56.176501
Worst Trade [%]                     -88.65117
Avg. Trade [%]                    

In [5]:
GOOG

,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900
...,...,...,...,...,...
2013-02-25,802.30,808.41,790.49,790.77,2303900
2013-02-26,795.00,795.95,784.40,790.13,2202500
2013-02-27,794.80,804.75,791.11,799.78,2026100
2013-02-28,801.10,806.99,801.03,801.20,2265800


In [6]:
from ta import add_all_ta_features
from backtesting.test import GOOG
import pandas as pd

In [7]:
# data = pd.read_csv("data/BAJAJ-AUTO.NS_1h.csv", parse_dates=True, index_col="Datetime")
# # rename columns to first letter caps
# data.columns = [col.capitalize() for col in data.columns]
# # sort by date
# data = data.sort_index()

# data

In [8]:
new_df = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume")

/opt/homebrew/lib/python3.11/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/opt/homebrew/lib/python3.11/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


In [9]:
# new_df.columns

In [10]:
new_df["return_2d"] = (new_df["Close"] / new_df["Open"]) - 1
new_df["return_prev_day"] = new_df["return_2d"].shift(1)
new_df["day_of_week"] = new_df.index.dayofweek
new_df["return_1w"] = (new_df["Close"] / new_df["Open"].shift(5)) - 1

new_df.sort_index(inplace=True)



In [11]:
new_df.columns

Index(['Series', 'Open', 'High', 'Low', 'Prev. close', 'Ltp', 'Close', 'Vwap',
       '52w h', '52w l',
       ...
       'momentum_pvo_signal', 'momentum_pvo_hist', 'momentum_kama',
       'others_dr', 'others_dlr', 'others_cr', 'return_2d', 'return_prev_day',
       'day_of_week', 'return_1w'],
      dtype='object', length=104)

In [12]:
# new_df.corr()
# make a heatmap
import seaborn as sns
import matplotlib.pyplot as plt

# remove the columns that are not needed
new_df.drop(["Open", "High", "Low", "Close", "Adj Close", "Volume"], axis=1, inplace=True)


KeyError: "['Adj Close'] not found in axis"